In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
import numpy as np
SEEDS=42
np.random.seed(SEEDS)
tf.random.set_seed(SEEDS)

In [ ]:
training = ImageDataGenerator(rescale = 1. / 255)

train_dataset = training.flow_from_directory(
    '../input/skin-cancer-malignant-vs-benign/train/',
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary')
data_list = []
batch_index = 0

while batch_index <= train_dataset.batch_index:
    data = train_dataset.next()
    data_list.append(data[0])
    batch_index = batch_index + 1

# now, data_array is the numeric data of whole images
data_array = np.asarray(data_list)

In [ ]:
validation = ImageDataGenerator(rescale = 1. / 255)

val_dataset = training.flow_from_directory(
    '../input/skin-cancer-malignant-vs-benign/test/',
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary')
data_list = []
batch_index = 0

while batch_index <= val_dataset.batch_index:
    data = val_dataset.next()
    data_list.append(data[0])
    batch_index = batch_index + 1

# now, data_array is the numeric data of whole images
val_array = np.asarray(data_list, dtype=object)

In [ ]:
RESIZE_TO = 384
CROP_TO = 224
BATCH_SIZE = 16
STEPS_PER_EPOCH = (2637/32)
AUTO = tf.data.experimental.AUTOTUNE  # optimise the pipeline performance
NUM_CLASSES = 9
# number of classes
SCHEDULE_LENGTH = (
    500  # we will train on lower resolution images and will still attain good results
)
SCHEDULE_BOUNDARIES = [
    200,
    300,
    400,
] 
# more the dataset size the schedule length increase

In [ ]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub



In [ ]:
import tensorflow_hub as hub
bit_model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
bit_module = hub.KerasLayer(bit_model_url)

In [ ]:
from tensorflow import keras
class MyBiTModel(keras.Model):
    def __init__(self, num_classes, module, **kwargs):
        super().__init__(**kwargs)

        self.num_classes = num_classes
        self.head = keras.layers.Dense(num_classes, kernel_initializer="zeros")
        self.bit_model = module

    def call(self, images):
        bit_embedding = self.bit_model(images)
        return self.head(bit_embedding)


model = MyBiTModel(num_classes=NUM_CLASSES, module=bit_module)


In [ ]:
learning_rate = 0.003 * BATCH_SIZE / 512

# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=SCHEDULE_BOUNDARIES,
    values=[
        learning_rate,
        learning_rate * 0.1,
        learning_rate * 0.01,
        learning_rate * 0.001,
    ],
)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
train_callbacks = [
   keras.callbacks.EarlyStopping(
       monitor="val_accuracy", patience=2, restore_best_weights=True   )
 ]

In [ ]:
import time
start = time.time()
history = model.fit(
    train_dataset,
    batch_size=32,
    epochs=50,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_dataset,
    callbacks=train_callbacks
    
    
 
    
)
print("Total time: ", time.time() - start, "seconds")



In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'val_accuracy', 'train_loss', 'val_loss'], bbox_to_anchor =(0.65, 1.00))
plt.show()
plt.savefig("bit.png")